In [9]:
import pandas as pd
import jieba
import openpyxl

def strQ2B(ustring):
    rstring = ""
    for uchar in ustring:
        inside_code=ord(uchar)
        if inside_code == 12288:    #全角空格直接转换            
            inside_code = 32
        elif (inside_code >= 65281 and inside_code <= 65374): #全角字符（除空格）根据关系转化
            inside_code -= 65248
        rstring += chr(inside_code)
        rstring=rstring.replace("(","（").replace(")","）")
    return rstring

def cut(fn):
    p=[]
    temp=jieba.cut(fn)
    for t in temp:
        if len(t)>1 and t in model:
            p.append(t)
    return p

def list_contain(l1,l2,method=1,exc=[],inc=[]):
    if method==1:
        not_in=[l for l in l2 if l not in l1 and l not in exc]
    elif method==2:
        not_in=[l for l in l2 if l not in l1 and l in inc]
    else:
        raise("error")
    if len(not_in)>0:
        return not_in
    else:
        return False

model = gensim.models.Word2Vec.load('./data/zhiwiki_news.word2vec')
df=pd.read_excel("./账户路径稽核.xlsx")
df=df.where(df.notnull(), "")
df[["账户全称","托管人","投资/资产管理人","大额账户名称(默认)","大额开户行行名(默认)"]] = \
df[["账户全称","托管人","投资/资产管理人","大额账户名称(默认)","大额开户行行名(默认)"]].applymap(lambda x: strQ2B(x))
wb=openpyxl.load_workbook("./账户路径稽核.xlsx")
ws=wb["Sheet1"]
exc=['托管','理财产品','资产','专户','资金','理财','分行','国泰君安','业务','银行','工行','交行','资管','公司','计划','建行','投资','国寿','客户','财产','账户', \
   '上海','结算','清算','股份','自有','浦发银行','净值','总行','河南省','系统','人民币','过渡','有限公司','民生银行','商行','农商','款项','委托','系列','调拨', \
   '养老','合资','产品','招行','组合','中心','建设银行','农行','其他','浦发','一年','国际','江苏省','年定','交易','商业银行','人保','人寿保险', \
   '中金公司','保险','宏源','收益','信托','常州','代理','有限责任','专用','光大银行','深圳','营运','企业','理事会','工商银行','之一','债券','农村','集团', \
   '核心','资本','应付','款待','网点','表内','汇入','个人','往来','国君','工银','中行','待查','博时','寿险','中信','专项','中国','财产保险','景丰', \
   '财险','自贸区','大家','总部','保管','合作','证券','广发','基金','管理部','上清','保险产品','金融市场','农村信用','联社','股票','of','存放','同业', \
   '保本','人寿','应付款','上海市分行','外汇','存款','普通','待处理','厦门','潍坊','双赢','浙江','兴业','私人','清户','惠鑫','中国建设银行', \
   '营业部','海通','苏州','民生','金色','存单','泉心','安瑞','江北','新区','管乐','存出','询价','净额','短期','本级','资本金','长安','其它','用户','江苏','福鹿', \
   '光大','东方','全球','管理','邮储','信达','农业银行','洛肯','分公司','配置','金睿','养老金','山西','大额','云南','郑州','保障','定期','联信', \
   '自贸','分账','核算','单元','山东','秦皇岛','杭州','中京','财信','互换','华润','纯债','无锡','石家庄','周期','出售','变动','保证金','商业银行总行','业务部','安心', \
   '得利','电力','社会保险','通海','邓州市','北京分行','策略','平定','宁波','收款','中车','运营部','投信','集中','盐亭','英资','健康','乐享','河北','型基金', \
   '特殊','成都','房改','信托公司','吉林','特定','金平','县联社','汇丰','一对一','崇阳县','王玉清','定增','乌鲁木齐','保诚','大陆','中银','安康','月盈','平衡','无忧', \
   '人生','灵活','深圳市','信诚','天津','国债','安信','华夏银行','中信证券','长城','汇兑','汇差','阳光','渤海','景晖','小盘','柘城县','方正','东亚','机构', \
   '帐户','芙蓉','广东','政府','四川省','区域','长春','长治','利安人','融资','担保','吴江','行业','支持','民企','发展','金融','尊享','融券', \
   '现金','三个','嘉裕','划转','内待','试验区','晋商','乐山市','大连','福建','漳州','金彩','中融','稳定','富浦','添利','佛山','北京','泰康', \
   '孟连县','汉口','人民','中民','永丰','弘晟','西盟','合作社','天府','青海','西宁','中山','明珠','东证','招银','进宝','之朝','稠州','金定','达富','深国投','万家','太保',\
   '短债','包商','管家','钱包','现金管理','南宁','合肥','长三角','精选','泰州','委外','天弘','创新','混合','欧洲','机遇','私行','补充','重庆','太原', \
   '天理','周开','威海','证券公司','广东省','蒙商','开式','二期','中国银行','西安','北大街','支行','相互','济南','徐州']
inc=['集合','开放式','开放','定向','固定','全委','封闭','传统','单一','自营','封闭式','混合型','分红','专一','八号','二号','一号','一期','一款','第二期','壹号', \
     '五十六','八期','开型','事件驱动','增强型']
dic={}

method=2
for i in range(len(df)):
    if df.loc[i,"大额账户名称(默认)"] in df.loc[i,"账户全称"]:
        ws["O{}".format(i+2)].value="是"
    elif df.loc[i,"大额开户行行名(默认)"] in df.loc[i,"账户全称"]:
        ws["O{}".format(i+2)].value="是"
    elif "托管专户" in df.loc[i,"大额账户名称(默认)"] or "投资资产" in df.loc[i,"大额账户名称(默认)"]:
        ws["O{}".format(i+2)].value="是"
    else:
        if method==1:
            extra=list_contain(cut(df.loc[i,"账户全称"])+cut(df.loc[i,"投资/资产管理人"])+cut(df.loc[i,"托管人"]),cut(df.loc[i,"大额账户名称(默认)"]), \
                           method,exc=exc)
        elif method==2:
            extra=list_contain(cut(df.loc[i,"账户全称"])+cut(df.loc[i,"投资/资产管理人"])+cut(df.loc[i,"托管人"]),cut(df.loc[i,"大额账户名称(默认)"]), \
                           method,inc=inc)
        if not extra:
            ws["O{}".format(i+2)].value="是"
        else:
            ws["P{}".format(i+2)].value=' '.join(extra)
wb.save("./账户路径稽核.xlsx")
# [i[0] for i in sorted(list(zip(dic.keys(),dic.values())),key=lambda x: x[1],reverse=True)]

/Users/lvjiaxin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).


['集合',
 '开放式',
 '开放',
 '定向',
 '固定',
 '自营',
 '全委',
 '封闭',
 '传统',
 '一号',
 '单一',
 '分红',
 '封闭式',
 '一期',
 '八号',
 '二号',
 '五十六',
 '一款',
 '第二期',
 '混合型',
 '壹号',
 '专一',
 '八期',
 '开型',
 '事件驱动',
 '增强型']